In [108]:
import random
from pulp import *
from collections import deque
import time

# Ucitavanje grafa

In [109]:
def load_graph_from_mtx(filename):
    graph = {}
    with open(filename, 'r') as file:
        for i, line in enumerate(file):
            # preskocimo prva tri reda (jer su tu podaci o broju grana, cvorova...)
            if i < 3:
                continue
            
            nodes = line.strip().split()
            node1, node2 = int(nodes[0]), int(nodes[1])

            if node1 not in graph:
                graph[node1] = set()
            if node2 not in graph:
                graph[node2] = set()

            graph[node1].add(node2)
            graph[node2].add(node1)

    return graph


# Generisanje random burning sekvenci

In [110]:

def generate_random_burning_sequence(graph):
    
    S=[] # sekvenca zapaljenih cvoreva
    B= set() # skup zapaljenih cvoreva
    NB= set(graph.keys()) # skup nezapaljenih cvoreva, inicajlno su to svi cvorevi grafa
    
    # ako imamo vise kandidata, biramo onog sa najvisim betweenness
    
    
    current_node = random.choice(list(NB))
    
   
    S.append(current_node)
    B.add(current_node)
    NB.remove(current_node)
    
    while len(B)<len(graph):
       
        # korak 2: zapaliti sve susjede (koji nisu zapaljeni) od cvorova u skupu zapaljenih, dodajemo ih u B, brisemo iz NB
        
        # Provjera: ako su svi cvorovi u NB zapravo cvorovi koji ce biti zapaljeni u narednom koraku (jer su direktni susjedi
        # zapaljenom cvoru nekom), onda cemo izabrati neki random cvor iz NB cisto kao simbol jos jednog koraka
        
        FB = []
        for b in B:
            neighbors_b1 = graph[b]
            for neighbor in neighbors_b1:
                if neighbor in NB:
                    if neighbor not in FB:
                        FB.append(neighbor)
             
        if set(FB) == NB:
            random_node = random.choice(list(NB))
            S.append(random_node)
          
        newlyburned = []
        for node in B:
            neighbors = graph[node]  # svi susjedi trenutnog zapaljenog cvora
            for neighbor in neighbors:
                if neighbor in NB:  # ako susjed nije zapaljen
                    newlyburned.append(neighbor)  # dodajemo ga u zapaljene
                    NB.remove(neighbor)  
        
        for new in newlyburned:
            B.add(new) #dodajemo sve susjede u zapaljene
        
        # pravimo listu onih cvorova za koje znamo da ce se u iducem koraku zapaliti. Bolje da izaberemo sljedeci direktni
        # cvor za koji znamo da nece biti zapaljen u narednom koraku
        
        FB2 = []
        for b in B:
            neighbors_b = graph[b]
            for neighbor in neighbors_b:
                if neighbor in NB:
                    if neighbor not in FB2:
                        FB2.append(neighbor)
       
        # korak 3: pronaci iduci cvor koji ce biti direktno zapaljen, dodati ga u S, B i izbrisati iz NB
        
        max_burned = -1
        best_node = None
        candidates_for_best = []
        
        if NB:
            for node in NB:
                if node not in FB2: 
                    candidates_for_best.append(node)
            if candidates_for_best:
                best_node = random.choice(candidates_for_best)
            else:
                best_node = None        
                
            if best_node is not None:
                S.append(best_node)
                B.add(best_node)
                NB.remove(best_node)
            else:
             # Ako nema nijednog najboljeg cvora znaci da su svi preostali cvorovi povezani i mogu biti zapaljeni
             # Dodajemo slucajni cvor iz preostalih nezapaljenih čvorova
                random_node = random.choice(list(NB))
                S.append(random_node)
                B.add(random_node)
                NB.remove(random_node)
           
            
        else:
            break
           
    return S
        
        

def generate_multiple_random_burning_sequences(graph, num_sequences):
    
    sequences = []
    for _ in range(num_sequences):
        sequence = generate_random_burning_sequence(graph)
        if sequence is not None:  
            sequences.append(sequence)
    return sequences


filename = "grafovi/manji/polbooks.mtx"
graph = load_graph_from_mtx(filename)


#max_burn_nodes = len(graph) // 2
num_sequences = 30  

burning_sequences = generate_multiple_random_burning_sequences(graph, num_sequences)

# Ispis rezultata
print(f"Generisane burning sekvence:")
for seq in burning_sequences:
    print(seq,"len:", len(seq))


Generisane burning sekvence:
[71, 17, 42, 44, 57] len: 5
[82, 21, 56, 37, 105] len: 5
[93, 46, 1, 16, 56] len: 5
[51, 55, 103, 61, 63] len: 5
[55, 94, 79, 88, 105] len: 5
[24, 72, 61, 104, 70] len: 5
[61, 33, 51, 2, 49] len: 5
[21, 92, 98, 104, 68] len: 5
[97, 41, 20, 51, 3] len: 5
[90, 11, 3, 46] len: 4
[30, 35, 95, 66, 104] len: 5
[96, 57, 55, 54, 105] len: 5
[93, 54, 51, 38, 18] len: 5
[27, 63, 68, 71, 96] len: 5
[72, 58, 24, 35, 105] len: 5
[21, 3, 62, 104, 63] len: 5
[1, 38, 65, 64, 96] len: 5
[1, 72, 63, 102, 70] len: 5
[104, 15, 103, 63, 64] len: 5
[100, 27, 49, 35, 39] len: 5
[79, 24, 103, 57, 51] len: 5
[43, 81, 105, 96, 66] len: 5
[45, 17, 98, 104, 64, 79] len: 6
[71, 17, 35, 45, 42] len: 5
[60, 99, 1, 45, 55, 33] len: 6
[33, 52, 79, 61, 102, 62] len: 6
[52, 6, 64, 35, 79] len: 5
[42, 99, 64, 104, 29] len: 5
[50, 47, 64, 35, 104] len: 5
[60, 88, 11, 45, 69] len: 5


In [111]:

def generate_distance_matrix(G):
    
    n = len(G)
    nodes = list(G.keys())  # Lista svih čvorova
    index_map = {v: i for i, v in enumerate(nodes)}  # Mapiranje čvorova na indekse
    
    # Inicijalizacija matrice udaljenosti sa "beskonačnim" vrednostima
    d = [[float('inf')] * n for _ in range(n)]
    
    # BFS za svaki čvor kako bismo pronašli udaljenosti do svih drugih čvorova
    for start in nodes:
        start_index = index_map[start]
        d[start_index][start_index] = 0  # Udaljenost do samog sebe je 0
        queue = deque([start])
        while queue:
            current = queue.popleft()
            current_index = index_map[current]
            for neighbor in G[current]:
                neighbor_index = index_map[neighbor]
                if d[start_index][neighbor_index] == float('inf'):  # Ako nije posećen
                    d[start_index][neighbor_index] = d[start_index][current_index] + 1
                    queue.append(neighbor)
    
    return d

def ILP_Pulp(L, k, G, burning_sequences=None):
    
    n = len(G)  
    d = generate_distance_matrix(G) 
    
    
    m = LpProblem("Graph_burning", LpMinimize)

    # binarne promjenljive
    b = [LpVariable(f"b_{j}", cat='Binary') for j in range(n)]
    x = [[LpVariable(f"x_{r}_{i}", cat='Binary') for i in range(n)] for r in range(k)]

    # funkcija cilja
    m += lpSum(x[r][i] for r in range(k) for i in range(n))

    
    # ogranicenje (14)
    for r in range(k):
        m += lpSum(x[r][j] for j in range(n)) <= 1

    # ogranicenje 15
    for j in range(n):
        suma = 0
        for i in range(k):
            for kk in range(n):
                if kk >= j:
                    if d[kk][j] <= i:
                        suma += x[i][kk]
                else:
                    if d[j][kk] <= i:
                        suma += x[i][kk]
        m += b[j] <= suma

    # ogranicenje 13
    for i in range(k):
        sum2 = 0
        sum1 = 1 if i == 0 else 0
        for j in range(n):
            sum2 += x[i][j]
            if i >= 1:
                sum1 += x[i-1][j]
        m += sum2 <= sum1

    # ogranicenje (16)
    m += lpSum(b) == n

    # provjera svake sekvence iz liste `burning_sequences`
    best_sequence = None
    best_obj_value = float('inf')
    results = []
    if burning_sequences:
        for seq in burning_sequences:
            m_copy = m.deepcopy() 

            # ogranicenja za trenutnu sekvencu
            for r, node in enumerate(seq):
                for i in range(n):
                    if i == node:
                        m_copy += x[r][i] == 1
                    else:
                        m_copy += x[r][i] == 0
            
           
            m_copy.solve()
            status = LpStatus[m_copy.status]
            obj_value = value(m_copy.objective)
            
            
            results.append({
                "sequence": seq,
                "status": status,
                "objective_value": obj_value
            })
            
            # azuriraj najbolju sekvencu
            if status == "Optimal" and obj_value < best_obj_value:
                best_obj_value = obj_value
                best_sequence = seq
    
    return results, best_sequence, best_obj_value

startTime = time.time()

k=10

results, best_sequence, best_obj_value = ILP_Pulp(LpMinimize, k, graph, burning_sequences=burning_sequences)


# Ispis najbolje sekvence
print("\nBest Sequence:")
print(f"Sequence: {best_sequence}")
print(f"Objective Value: {best_obj_value}")

print("--- %s sekundi ---" % (time.time() - startTime))



Best Sequence:
Sequence: [61, 33, 51, 2, 49]
Objective Value: 5.0
--- 10.396966695785522 sekundi ---
